In [3]:
# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# 2) Install / upgrade required libs
!pip install -q --upgrade pip
!pip install -q git+https://github.com/huggingface/diffusers.git transformers accelerate safetensors ftfy
!pip install -q huggingface-hub


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
from huggingface_hub import login
# Paste your Hugging Face token when prompted
login()

In [5]:
import torch
from diffusers import StableDiffusionPipeline

model_id = "runwayml/stable-diffusion-v1-5"   # reliable, open-source
device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

# optional: enable memory-efficient attention if available
try:
    pipe.enable_xformers_memory_efficient_attention()
except Exception:
    pass

print("Model loaded. Device:", device)


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Model loaded. Device: cuda


In [6]:
import os, time
from PIL import Image
import torch

# Output folder on Drive
OUTDIR = "/content/drive/MyDrive/SD_outputs"
os.makedirs(OUTDIR, exist_ok=True)

# Seeds to make reproducible variations
seeds = [42, 1234, 2025]

# Prompts: change text if you want; these are tuned for "different angles"
locations = {
    "bandra_worli_sea_link": [
        "Ultra-realistic aerial drone shot of Bandra-Worli Sea Link, wide angle, golden hour, cinematic, high detail",
        "Side coastline view of Bandra-Worli Sea Link, midday, wide shot, clear sky, photographic realism",
        "Night long-exposure of Bandra-Worli Sea Link with light trails and reflections on the Arabian Sea, high detail"
    ],
    "hawa_mahal_jaipur": [
        "Front façade of Hawa Mahal, Jaipur, golden hour, warm light, highly detailed sandstone texture, minimal crowd",
        "Wide-angle side view of Hawa Mahal with street foreground, clear sky, high-resolution architectural photo",
        "Close-up of Hawa Mahal windows and carvings, shallow depth of field, warm sunlight"
    ],
    "nagpur_rainforest": [
        "Dense Nagpur rainforest early morning, mist rising between trees, lush green, cinematic atmosphere",
        "Trail through Nagpur rainforest, low-angle view with canopy, damp soil and foliage, high detail",
        "Waterfall inside Nagpur rainforest, misty light, rich vegetation, wide shot, natural lighting"
    ]
}

# If you only need exactly 6 images for your assignment, take any 2 locations and 3 prompts each,
# or change the locations dict accordingly.

# Generate
guidance_scale = 7.5
steps = 28
height = 512
width = 512

for loc, plist in locations.items():
    # generate up to 3 images per location (modify if you want)
    for i, prompt in enumerate(plist[:3]):  # [:3] ensures 3 per location
        seed = seeds[i % len(seeds)]
        gen = torch.Generator(device).manual_seed(seed)
        out_name = f"{loc}_{i+1}_seed{seed}.png"
        out_path = os.path.join(OUTDIR, out_name)
        print("Generating:", out_name, "prompt:", prompt)
        image = pipe(prompt, num_inference_steps=steps, guidance_scale=guidance_scale,
                     height=height, width=width, generator=gen).images[0]
        image.save(out_path)
        print("Saved:", out_path)
        time.sleep(1)


Generating: bandra_worli_sea_link_1_seed42.png prompt: Ultra-realistic aerial drone shot of Bandra-Worli Sea Link, wide angle, golden hour, cinematic, high detail


  0%|          | 0/28 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/SD_outputs/bandra_worli_sea_link_1_seed42.png
Generating: bandra_worli_sea_link_2_seed1234.png prompt: Side coastline view of Bandra-Worli Sea Link, midday, wide shot, clear sky, photographic realism


  0%|          | 0/28 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/SD_outputs/bandra_worli_sea_link_2_seed1234.png
Generating: bandra_worli_sea_link_3_seed2025.png prompt: Night long-exposure of Bandra-Worli Sea Link with light trails and reflections on the Arabian Sea, high detail


...

Saved: /content/drive/MyDrive/SD_outputs/nagpur_rainforest_1_seed42.png
Generating: nagpur_rainforest_2_seed1234.png prompt: Trail through Nagpur rainforest, low-angle view with canopy, damp soil and foliage, high detail


  0%|          | 0/28 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/SD_outputs/nagpur_rainforest_2_seed1234.png
Generating: nagpur_rainforest_3_seed2025.png prompt: Waterfall inside Nagpur rainforest, misty light, rich vegetation, wide shot, natural lighting


  0%|          | 0/28 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/SD_outputs/nagpur_rainforest_3_seed2025.png


In [14]:
prompt = ("Photorealistic view of the Bandra–Worli Sea Link at sunset,soft pink and purple evening sky with light haze, taken from the shoreline with foreground rocks and calm water, long-exposure smooth water effect, high-resolution DSLR photograph, cinematic composition.")

image = pipe(prompt,
             num_inference_steps=30,
             guidance_scale=7.5,
             height=1024,
             width=1024,
             generator=torch.Generator(device).manual_seed(42)
            ).images[0]
image.save("/content/drive/MyDrive/SD_outputs/sea_link_2.png")


  0%|          | 0/30 [00:00<?, ?it/s]